# Rule merging 

---

## 1. Importing necessary modules

In [32]:
from utils.N01_agent_setup import (
    # Fact classes for our rule engine
    TelemetryFact,
    DegradationFact,
    GapFact,
    RadioFact,
    RaceStatusFact,
    StrategyRecommendation,
    F1StrategyEngine,
    
    # Utility functions for data transformation
    transform_tire_predictions,
    load_tire_predictions,
    transform_lap_time_predictions,
    load_lap_time_predictions,
    transform_radio_analysis,
    process_radio_message,
    transform_gap_data_with_consistency,
    load_gap_data,
    calculate_gap_consistency
)

# Import the rule engines from each domain
# Tire degradation rules
from utils.N02_degradation_time_rules import F1DegradationRules

# Lap time prediction rules
from utils.N03_lap_time_rules import F1LapTimeRules

# Radio communication analysis rules
from utils.N04_nlp_rules import F1RadioRules  

# Gap analysis rules
from utils.N05_gap_rules import F1GapRules

# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import sys
import json
import time

# Import Experta components
from experta import Rule, NOT, OR, AND, AS, MATCH, TEST, EXISTS
from experta import DefFacts, Fact, Field, KnowledgeEngine

# Add parent directory to path to access modules
sys.path.append(os.path.abspath('../'))

# Configure visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("notebook", font_scale=1.2)

print("All components imported successfully!")

All components imported successfully!


---

## 2. Main Engine Class

This class inherits from all the enignes defined for each topic, and also a conflic resolution capability to avoid contradictory strategies being fired at the same time.

In [33]:
class F1CompleteStrategyEngine(F1DegradationRules, F1LapTimeRules, F1RadioRules, F1GapRules):
    """
    Unified strategy engine that integrates all rule systems:
    - Tire degradation rules 
    - Lap time prediction rules
    - Radio communication analysis rules
    - Gap analysis rules
    
    This class inherits from all specialized rule engines to combine their rules
    and adds conflict resolution capabilities.
    """
    
    def __init__(self):
        """Initialize the integrated engine"""
        # Call the parent constructor
        super().__init__()
        # Track which rule systems have fired rules
        self.active_systems = {
            'degradation': False,
            'lap_time': False,
            'radio': False,
            'gap': False
        }
    
    def get_recommendations(self):
        """
        Get all recommendations from the rule engine with enhanced conflict resolution.
        
        Returns:
            list: Sorted list of recommendations with conflicts resolved
        """
        # Get the base recommendations from parent method
        all_recommendations = super().get_recommendations()
        
        # If we have very few recommendations, no need for complex conflict resolution
        if len(all_recommendations) <= 2:
            return all_recommendations
        
        # Group recommendations by driver
        driver_recommendations = {}
        for rec in all_recommendations:
            driver = rec.get('DriverNumber', 0)  # Default to 0 if driver is not specified
            if driver not in driver_recommendations:
                driver_recommendations[driver] = []
            driver_recommendations[driver].append(rec)
        
        # Process each driver's recommendations for conflicts
        resolved_recommendations = []
        for driver, recs in driver_recommendations.items():
            # Only need conflict resolution if multiple recommendations
            if len(recs) > 1:
                resolved = self._resolve_conflicts(recs)
                resolved_recommendations.extend(resolved)
            else:
                # Single recommendation, no conflicts to resolve
                resolved_recommendations.extend(recs)
        
        # Sort by priority and confidence
        return sorted(
            resolved_recommendations,
            key=lambda x: (x.get('priority', 0), x.get('confidence', 0)),
            reverse=True
        )
    
    def _resolve_conflicts(self, recommendations):
        """
        Resolve conflicts between recommendations for the same driver.
        
        This method looks for contradictory recommendations and resolves them
        based on priority, confidence, and the nature of the conflict.
        
        Args:
            recommendations: List of recommendations for a single driver
            
        Returns:
            list: Resolved list of recommendations
        """
        # Group recommendations by action type
        action_groups = {}
        for rec in recommendations:
            action = rec['action']
            if action not in action_groups:
                action_groups[action] = []
            action_groups[action].append(rec)
        
        # Define conflicting action pairs
        conflicting_pairs = [
            # Can't extend stint and pit at the same time
            ('extend_stint', 'pit_stop'),
            ('extend_stint', 'prioritize_pit'),
            ('extend_stint', 'defensive_pit'),
            ('extend_stint', 'consider_pit'),
            
            # No need for preparation if immediate pit is recommended
            ('prepare_pit', 'pit_stop'),
            ('prepare_pit', 'prioritize_pit'),
            
            # Can't do undercut and overcut at the same time
            ('perform_undercut', 'perform_overcut')
        ]
        
        # Check for each conflict pair
        resolved = []
        excluded_recommendations = set()
        
        for action1, action2 in conflicting_pairs:
            if action1 in action_groups and action2 in action_groups:
                # We have a conflict!
                group1 = action_groups[action1]
                group2 = action_groups[action2]
                
                # Get the highest priority/confidence recommendation from each group
                best1 = max(group1, key=lambda x: (x.get('priority', 0), x.get('confidence', 0)))
                best2 = max(group2, key=lambda x: (x.get('priority', 0), x.get('confidence', 0)))
                
                # Compare and keep only the better one
                if (best1.get('priority', 0), best1.get('confidence', 0)) >= (best2.get('priority', 0), best2.get('confidence', 0)):
                    # best1 wins, exclude all from group2
                    excluded_recommendations.update(id(r) for r in group2)
                else:
                    # best2 wins, exclude all from group1
                    excluded_recommendations.update(id(r) for r in group1)
        
        # Add non-excluded recommendations
        for rec in recommendations:
            if id(rec) not in excluded_recommendations:
                resolved.append(rec)
        
        # Enhance the winning recommendations with context from conflicting ones
        if len(resolved) < len(recommendations):
            # We had conflicts and resolved them
            for rec in resolved:
                rec['explanation'] += " (Selected as optimal strategy after resolving conflicts)"
        
        return resolved
    
    def record_rule_fired(self, rule_name):
        """
        Record which rule fired and track which rule system it belongs to.
        
        Args:
            rule_name: Name of the rule that fired
        """
        # Standard recording from parent class
        super().record_rule_fired(rule_name)
        
        # Also track which system the rule belongs to
        if rule_name.startswith(('high_degradation', 'stint_extension', 'early_degradation')):
            self.active_systems['degradation'] = True
        elif rule_name.startswith(('optimal_performance', 'performance_cliff', 'post_traffic')):
            self.active_systems['lap_time'] = True
        elif rule_name.startswith(('grip_issue', 'weather_information', 'incident_reaction')):
            self.active_systems['radio'] = True
        elif rule_name.startswith(('undercut_opportunity', 'defensive_pit', 'strategic_overcut')):
            self.active_systems['gap'] = True

### 2.1 Testing class initialization

In [34]:
# Test that our integrated engine initializes correctly
try:
    engine = F1CompleteStrategyEngine()
    print("✅ Successfully created the integrated engine!")
    
    # Use reset to initialize the engine's working memory
    engine.reset()
    print("✅ Engine reset successful!")
    
    # Check MRO (Method Resolution Order) to confirm correct inheritance
    print("\nMethod Resolution Order:")
    for cls in F1CompleteStrategyEngine.__mro__:
        print(f"  - {cls.__name__}")
    
    # Print info about active systems
    print("\nActive rule systems:", engine.active_systems)
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

✅ Successfully created the integrated engine!
✅ Engine reset successful!

Method Resolution Order:
  - F1CompleteStrategyEngine
  - F1DegradationRules
  - F1LapTimeRules
  - F1RadioRules
  - F1GapRules
  - F1StrategyEngine
  - KnowledgeEngine
  - object

Active rule systems: {'degradation': False, 'lap_time': False, 'radio': False, 'gap': False}


---

## 3. Transformation Function for integrating all the facts



In [35]:
from utils.N01_agent_setup import calculate_race_phase

def transform_all_facts(driver_number, tire_predictions=None, lap_predictions=None, 
                       gap_data=None, radio_json_path=None, current_lap=None, 
                       total_laps=66):
    """
    Transform all data sources into facts for the integrated rule engine.
    
    This function combines fact transformation from all domains:
    - Tire degradation
    - Lap time prediction
    - Gap analysis
    - Radio communications
    
    Args:
        driver_number: Driver to focus on
        tire_predictions: DataFrame with tire degradation predictions
        lap_predictions: DataFrame with lap time predictions
        gap_data: DataFrame with gap information
        radio_json_path: Path to radio analysis JSON
        current_lap: Current race lap
        total_laps: Total race laps
        
    Returns:
        Dictionary of facts to declare in the engine
    """
    facts = {}
    
    # 1. Transform tire degradation data
    if tire_predictions is not None:
        try:
            tire_facts = transform_tire_predictions(tire_predictions, driver_number)
            if tire_facts:
                # Ensure values are valid for the Fact schema
                if 'degradation' in tire_facts and tire_facts['degradation'] is not None:
                    # For DegradationFact, ensure no None values in required fields
                    deg_fact = tire_facts['degradation']
                    for field_name in ['degradation_rate']:
                        if field_name in deg_fact and deg_fact[field_name] is None:
                            deg_fact[field_name] = 0.0  # Default to 0.0 for None values
                
                # For TelemetryFact, ensure no None values in required fields
                if 'telemetry' in tire_facts and tire_facts['telemetry'] is not None:
                    telemetry_fact = tire_facts['telemetry']
                    for field_name in ['tire_age', 'compound_id', 'driver_number', 'position']:
                        if field_name in telemetry_fact and telemetry_fact[field_name] is None:
                            if field_name in ['tire_age', 'compound_id', 'position']:
                                telemetry_fact[field_name] = 0  # Default to 0 for None values in int fields
                            elif field_name == 'driver_number':
                                telemetry_fact[field_name] = driver_number  # Default to the driver number
                
                facts.update(tire_facts)
                print(f"✓ Transformed tire degradation data for Driver #{driver_number}")
        except Exception as e:
            print(f"✗ Error transforming tire data: {str(e)}")
    
    # 2. Transform lap time data
    if lap_predictions is not None:
        try:
            lap_facts = transform_lap_time_predictions(lap_predictions, driver_number)
            if lap_facts:
                # Handle None values in telemetry fact to avoid schema validation errors
                if 'telemetry' in lap_facts and lap_facts['telemetry'] is not None:
                    telemetry_fact = lap_facts['telemetry']
                    # Ensure lap_time field has a valid float (not None)
                    if 'lap_time' in telemetry_fact and telemetry_fact['lap_time'] is None:
                        # Use predicted_lap_time as fallback, or 0.0 if that's None too
                        if telemetry_fact.get('predicted_lap_time') is not None:
                            telemetry_fact['lap_time'] = telemetry_fact['predicted_lap_time']
                        else:
                            telemetry_fact['lap_time'] = 0.0
                    
                    # Ensure other required fields have valid values
                    for field_name in ['predicted_lap_time', 'compound_id', 'tire_age', 'position']:
                        if field_name in telemetry_fact and telemetry_fact[field_name] is None:
                            # Default values for any None fields
                            if field_name == 'predicted_lap_time':
                                telemetry_fact[field_name] = telemetry_fact.get('lap_time', 0.0)
                            elif field_name in ['compound_id', 'tire_age', 'position']:
                                telemetry_fact[field_name] = 0
                
                facts.update(lap_facts)
                print(f"✓ Transformed lap time data for Driver #{driver_number}")
        except Exception as e:
            print(f"✗ Error transforming lap time data: {str(e)}")
    
    # 3. Transform gap data
    if gap_data is not None:
        try:
            # Ensure gap consistency is calculated
            if 'consistent_gap_ahead_laps' not in gap_data.columns:
                gap_data = calculate_gap_consistency(gap_data)
            gap_fact = transform_gap_data_with_consistency(gap_data, driver_number)
            if gap_fact:
                # Handle None values in gap fact to avoid schema validation errors
                for field_name in ['gap_ahead', 'gap_behind', 'car_ahead', 'car_behind']:
                    if field_name in gap_fact and gap_fact[field_name] is None:
                        if field_name in ['gap_ahead', 'gap_behind']:
                            gap_fact[field_name] = 0.0  # Default to 0.0 for distance gaps
                        else:
                            gap_fact[field_name] = 0  # Default to 0 for car numbers
                
                facts['gap'] = gap_fact
                print(f"✓ Transformed gap data for Driver #{driver_number}")
        except Exception as e:
            print(f"✗ Error transforming gap data: {str(e)}")
    
    # 4. Transform radio data
    if radio_json_path:
        try:
            radio_fact = transform_radio_analysis(radio_json_path)
            if radio_fact:
                facts['radio'] = radio_fact
                print(f"✓ Transformed radio communication data")
        except Exception as e:
            print(f"✗ Error transforming radio data: {str(e)}")
    
    # 5. Create race status fact (always required)
    try:
        from utils.N01_agent_setup import calculate_race_phase
        race_phase = calculate_race_phase(current_lap, total_laps)
        facts['race_status'] = RaceStatusFact(
            lap=current_lap, 
            total_laps=total_laps, 
            race_phase=race_phase, 
            track_status="clear"
        )
        print(f"✓ Created race status fact: Lap {current_lap}/{total_laps} ({race_phase})")
    except Exception as e:
        print(f"✗ Error creating race status fact: {str(e)}")
        # Provide fallback race status
        facts['race_status'] = RaceStatusFact(
            lap=current_lap or 1,
            total_laps=total_laps,
            race_phase="mid",
            track_status="clear"
        )
    
    return facts

---

## 4. Load and prepare main function


In [36]:
def load_all_data(race_data_path, models_path=None, lap_model_path=None, gap_data_path=None, radio_message=None):
    """
    Load all necessary data for the strategy engine from various sources.
    
    Args:
        race_data_path: Path to race telemetry CSV
        models_path: Path to tire prediction models directory
        lap_model_path: Path to lap time prediction model file
        gap_data_path: Optional path to gap data CSV
        radio_message: Optional radio message text to analyze
        
    Returns:
        Dictionary with all loaded data
    """
    result = {}
    
    # 1. Load race telemetry data (required)
    print("Loading race telemetry data...")
    try:
        race_data = pd.read_csv(race_data_path)
        result['race_data'] = race_data
        print(f"✓ Loaded race data: {len(race_data)} rows")
    except Exception as e:
        print(f"✗ Could not load race data: {str(e)}")
        return result  # Cannot continue without race data
    
    # 2. Generate tire degradation predictions
    if models_path:
        print("Generating tire degradation predictions...")
        try:
            # Default monitoring thresholds by compound
            compound_thresholds = {1: 6, 2: 12, 3: 25}
            tire_predictions = load_tire_predictions(
                race_data, 
                models_path, 
                compound_thresholds=compound_thresholds
            )
            result['tire_predictions'] = tire_predictions
            print(f"✓ Generated tire predictions: {len(tire_predictions) if tire_predictions is not None else 0} rows")
        except Exception as e:
            print(f"✗ Could not generate tire predictions: {str(e)}")
    
    # 3. Generate lap time predictions with a separate model path
    if lap_model_path:
        print("Generating lap time predictions...")
        try:
            # Import the lap prediction module directly
            sys.path.append(os.path.abspath('../'))
            from ML_tyre_pred.ML_utils.N00_model_lap_prediction import predict_lap_times
            
            # Use the function directly instead of the wrapper
            lap_predictions = predict_lap_times(
                race_data,
                model_path=lap_model_path,
                include_next_lap=True
            )
            
            result['lap_predictions'] = lap_predictions
            print(f"✓ Generated lap time predictions: {len(lap_predictions) if lap_predictions is not None else 0} rows")
        except Exception as e:
            print(f"✗ Could not generate lap time predictions: {str(e)}")
    
    # 4. Create gap data from race data if not provided as CSV
    if gap_data_path:
        print(f"Loading gap data from {gap_data_path}...")
        try:
            gap_data = pd.read_csv(gap_data_path)
            result['gap_data'] = gap_data
            print(f"✓ Loaded gap data: {len(gap_data)} rows")
        except Exception as e:
            print(f"✗ Could not load gap data: {str(e)}")
    
    # 5. Process radio message if provided
    if radio_message:
        print(f"Processing radio message: '{radio_message}'...")
        try:
            json_path = process_radio_message(radio_message)
            if json_path:
                result['radio_json_path'] = json_path
                print(f"✓ Processed radio message to: {json_path}")
        except Exception as e:
            print(f"✗ Could not process radio message: {str(e)}")
    
    return result

## 5. Complete end-to-end pipeline

In [37]:
def analyze_strategy(
    driver_number, 
    race_data_path, 
    models_path=None,
    lap_model_path=None,
    gap_data_path=None,
    radio_message=None, 
    current_lap=None, 
    total_laps=66
):
    """
    Complete end-to-end F1 strategy analysis pipeline.
    
    This function integrates all components of the F1 strategy system:
    1. Loads and prepares data from all sources
    2. Transforms data into facts for the rule engine
    3. Runs the integrated engine with all rule systems
    4. Returns prioritized strategy recommendations
    
    Args:
        driver_number: Driver number to analyze
        race_data_path: Path to race telemetry CSV
        models_path: Path to tire prediction models
        lap_model_path: Path to lap time prediction model
        gap_data_path: Optional path to gap data CSV
        radio_message: Optional radio message to analyze
        current_lap: Current lap number (defaults to max in data)
        total_laps: Total race laps
        
    Returns:
        List of strategy recommendations
    """
    start_time = time.time()
    print(f"\n{'='*80}")
    print(f"F1 STRATEGY ANALYSIS FOR DRIVER #{driver_number}")
    print(f"{'='*80}")
    
    # Step 1: Load all data sources
    print("\n--- LOADING DATA ---")
    data = load_all_data(race_data_path, models_path, lap_model_path, gap_data_path, radio_message)
    
    # Determine current lap if not provided
    if current_lap is None and 'race_data' in data:
        try:
            driver_data = data['race_data'][data['race_data']['DriverNumber'] == driver_number]
            if not driver_data.empty:
                if 'LapNumber' in driver_data.columns:
                    current_lap = int(driver_data['LapNumber'].max())
                else:
                    # Use TyreAge as fallback
                    current_lap = int(driver_data['TyreAge'].max())
                print(f"✓ Determined current lap: {current_lap}")
            else:
                current_lap = 20  # Default mid-race
                print(f"✓ Using default lap: {current_lap}")
        except:
            current_lap = 20  # Default
            print(f"✓ Using default lap: {current_lap}")
    
    # Step 2: Transform data into facts
    print("\n--- TRANSFORMING DATA INTO FACTS ---")
    facts = transform_all_facts(
        driver_number=driver_number,
        tire_predictions=data.get('tire_predictions'),
        lap_predictions=data.get('lap_predictions'),
        gap_data=data.get('gap_data'),
        radio_json_path=data.get('radio_json_path'),
        current_lap=current_lap,
        total_laps=total_laps
    )
    
    # Step 3: Run the integrated strategy engine
    print("\n--- RUNNING INTEGRATED STRATEGY ENGINE ---")
    engine = F1CompleteStrategyEngine()
    engine.reset()
    
    # Declare facts to the engine
    for fact_type, fact in facts.items():
        if fact is not None:
            try:
                engine.declare(fact)
                print(f"✓ Declared {type(fact).__name__}")
            except Exception as e:
                print(f"✗ Error declaring {type(fact).__name__}: {str(e)}")
    
    # Run the engine - this will activate all applicable rules
    print("\nExecuting rules...")
    engine.run()
    
    # Get recommendations with conflict resolution
    recommendations = engine.get_recommendations()
    
    # Step 4: Show results
    print(f"\n--- ANALYSIS RESULTS ---")
    print(f"Generated {len(recommendations)} strategy recommendations")
    
    if recommendations:
        for i, rec in enumerate(recommendations):
            print(f"\nRecommendation {i+1}:")
            print(f"  Action: {rec['action']}")
            print(f"  Confidence: {rec['confidence']:.2f}")
            print(f"  Priority: {rec['priority']}")
            print(f"  Explanation: {rec['explanation']}")
    else:
        print("No recommendations generated. Try with different data inputs.")
    
    # Show which rule systems were activated
    print("\nActivated rule systems:")
    for system, active in engine.active_systems.items():
        status = "✓" if active else "✗"
        print(f"  {status} {system.capitalize()} rules")
    
    elapsed_time = time.time() - start_time
    print(f"\nAnalysis completed in {elapsed_time:.2f} seconds")
    
    return recommendations

---

In [38]:
def run_example_analysis():
    """
    Example demonstrating the complete strategy analysis pipeline.
    
    Uses the provided CSV files and a sample radio message to run
    a full strategy analysis for a specific driver.
    """
    print("\n=== RUNNING EXAMPLE STRATEGY ANALYSIS ===")
    
    # Define paths to data files
    race_data_path = '../../outputs/week3/lap_prediction_data.csv'
    models_path = '../../outputs/week5/models/'
    lap_model_path = '../../outputs/week3/xgb_sequential_model.pkl'  # Separate path for lap model
    
    # Sample radio message
    radio_message = "Box this lap for softs, there's rain expected in 10 minutes"
    
    # Driver to analyze (Lewis Hamilton)
    driver_number = 44
    
    # Run the analysis
    recommendations = analyze_strategy(
        driver_number=driver_number,
        race_data_path=race_data_path,
        models_path=models_path,
        lap_model_path=lap_model_path,
        radio_message=radio_message,
        current_lap=20,  # Mid-race scenario
        total_laps=66    # Typical F1 race length
    )
    
    return recommendations



In [39]:
# # Run the example if executed directly
# if __name__ == "__main__":
#     example_recommendations = run_example_analysis()

In [40]:
def analyze_all_drivers(
    race_data_path, 
    models_path=None,
    lap_model_path=None,
    output_path=None
):
    """
    Analyze strategy for all drivers at key moments in the race.
    
    Simplified function that processes every driver and generates
    recommendations for each at strategic points during the race.
    
    Args:
        race_data_path: Path to the telemetry CSV
        models_path: Path to the tire degradation models
        lap_model_path: Path to the lap time prediction model
        output_path: Path to save results (optional)
    
    Returns:
        DataFrame with all recommendations
    """
    print(f"\n{'='*80}")
    print("ANALYSIS FOR ALL DRIVERS")
    print(f"{'='*80}")
    
    # Load race data
    race_data = pd.read_csv(race_data_path)
    print(f"Data loaded: {len(race_data)} rows")
    
    # Initialize predictions
    tire_predictions = None
    lap_predictions = None
    
    # Generate tire predictions if models provided
    if models_path:
        try:
            compound_thresholds = {1: 6, 2: 12, 3: 25}
            tire_predictions = load_tire_predictions(race_data, models_path, compound_thresholds)
            count = len(tire_predictions) if tire_predictions is not None else 0
            print(f"Tire predictions generated: {count} rows")
        except Exception as e:
            print(f"Error generating tire predictions: {e}")
    
    # Generate lap time predictions if model provided
    if lap_model_path:
        try:
            lap_predictions = load_lap_time_predictions(race_data, lap_model_path)
            count = len(lap_predictions) if lap_predictions is not None else 0
            print(f"Lap time predictions generated: {count} rows")
        except Exception as e:
            print(f"Error generating lap time predictions: {e}")
    
    # Example radio message for all drivers
    radio_message = "Box this lap for softs, there's rain expected in 10 minutes"
    json_path = process_radio_message(radio_message)
    
    # Determine total laps
    total_laps = 66  # Default value
    if 'LapNumber' in race_data.columns:
        total_laps = int(race_data['LapNumber'].max())
    elif 'TyreAge' in race_data.columns:
        total_laps = int(race_data['TyreAge'].max())
    
    # Strategic points to analyze (25%, 50%, 75%)
    strategic_points = [
        int(total_laps * 0.25),
        int(total_laps * 0.5),
        int(total_laps * 0.75)
    ]
    
    # Process each driver
    all_recommendations = []
    drivers = sorted(race_data['DriverNumber'].unique())
    
    for driver_number in drivers:
        print(f"\nAnalyzing driver #{driver_number}")
        
        for lap in strategic_points:
            print(f"  Lap {lap}/{total_laps}")
            
            # Initialize the rules engine
            engine = F1CompleteStrategyEngine()
            engine.reset()
            
            # Declare tire-related facts
            if tire_predictions is not None:
                try:
                    tire_facts = transform_tire_predictions(tire_predictions, driver_number)
                    if tire_facts:
                        for fact_type in ['degradation', 'telemetry']:
                            if fact_type in tire_facts and tire_facts[fact_type] is not None:
                                fact = tire_facts[fact_type]
                                # Replace None with defaults
                                for field, value in fact.items():
                                    if value is None:
                                        fact[field] = 0.0 if isinstance(value, float) else 0
                                engine.declare(fact)
                except Exception as e:
                    print(f"    Error transforming tire data: {e}")
            
            # Declare lap-time-related facts
            if lap_predictions is not None:
                try:
                    lap_facts = transform_lap_time_predictions(lap_predictions, driver_number)
                    if lap_facts and 'telemetry' in lap_facts:
                        telemetry = lap_facts['telemetry']
                        # Fill missing lap_time from prediction if needed
                        if telemetry.get('lap_time') is None:
                            telemetry['lap_time'] = telemetry.get('predicted_lap_time', 0.0)
                        engine.declare(telemetry)
                except Exception as e:
                    print(f"    Error transforming lap time data: {e}")
            
            # Declare race status fact
            race_phase = (
                "start" if lap < total_laps * 0.25 
                else "end" if lap > total_laps * 0.75 
                else "mid"
            )
            race_status = RaceStatusFact(
                lap=lap,
                total_laps=total_laps,
                race_phase=race_phase,
                track_status="clear"
            )
            engine.declare(race_status)
            
            # Declare radio analysis fact if available
            if json_path:
                try:
                    radio_fact = transform_radio_analysis(json_path)
                    engine.declare(radio_fact)
                except Exception as e:
                    print(f"    Error transforming radio data: {e}")
            
            # Run rules engine and collect recommendations
            engine.run()
            recommendations = engine.get_recommendations()
            
            # Attach metadata to each recommendation
            for rec in recommendations:
                rec['DriverNumber'] = driver_number
                rec['LapNumber'] = lap
                rec['RacePhase'] = race_phase
            
            all_recommendations.extend(recommendations)
            print(f"    {len(recommendations)} recommendations generated")
    
    # Convert recommendations to DataFrame
    if all_recommendations:
        results_df = pd.DataFrame(all_recommendations)
        
        # Save to CSV if an output path is specified
        if output_path:
            results_df.to_csv(output_path, index=False)
            print(f"\nResults saved to: {output_path}")
        
        print(f"\nTotal recommendations: {len(results_df)}")
        return results_df
    else:
        print("No recommendations generated")
        return pd.DataFrame()


In [41]:
def run_all_drivers_analysis():
    """
    Example demonstrating the analysis of all drivers.
    """
    # Paths to data files
    race_data_path = '../../outputs/week3/lap_prediction_data.csv'
    models_path = '../../outputs/week5/models/'
    lap_model_path = '../../outputs/week3/xgb_sequential_model.pkl'
    output_path = '../../outputs/week6/enhanced_recommendations.csv'
    
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Run the analysis
    results = analyze_all_drivers(
        race_data_path=race_data_path,
        models_path=models_path,
        lap_model_path=lap_model_path,
        output_path=output_path
    )
    
    return results

# Execute when run as a script
if __name__ == "__main__":
    results = run_all_drivers_analysis()


ANALYSIS FOR ALL DRIVERS
Data loaded: 1180 rows
Data loaded and validated: 1180 rows, 15 columns
Processing Medium tires (ID: 2)...
Processing Hard tires (ID: 3)...
  No laps with new tires for Hard, using TyreAge=2.0 as baseline
Processing Soft tires (ID: 1)...
  No laps with new tires for Soft, using TyreAge=2.0 as baseline
Degradation metrics successfully calculated
Processed data format: 16 features
Created 339 sequences of 5 laps each
Sequences by compound: {1: 158, 2: 147, 3: 34}
Using device: cuda
Global model loaded from: ../../outputs/week5/models/tire_degradation_tcn.pth
Specialized model for compound 1 loaded
Specialized model for compound 2 loaded
Specialized model for compound 3 loaded
Models loaded: 1 global model and 3 specialized models
Prepared tensor for model: shape=torch.Size([339, 5, 16])
Generated ensemble predictions for 339 sequences
Formatted results: 1017 predictions for 20 drivers
Tire predictions generated: 1017 rows
Model loaded successfully with 25 featur